In [1]:
import sys 
sys.path.insert(0,'../')

In [5]:
from IO import*
from data import* 
from Unet_models import*
from segmentation import*

import h5py as h5
import patchify as patch
import cv2

from tqdm import tqdm

In [3]:
# check availible GPUS and set memory growth 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

2 Physical GPUs, 2 Logical GPUs


2022-03-21 11:32:31.636555: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-21 11:32:32.662115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14635 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:61:00.0, compute capability: 7.0
2022-03-21 11:32:32.663017: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14635 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:db:00.0, compute capability: 7.0


In [4]:
# set mixed precision (32/16 bit) presision
tf.keras.mixed_precision.experimental.set_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0


In [6]:
def Unet_segmentation(patch_data, model, batch_size):
    segmented = np.zeros(patch_data.shape, dtype = np.float32)
    # check noramalization
    if (patch_data.max() > 1):
        patch_data = patch_data / patch_data.max()
    # reshape patch data [nz,nx,ny,z,x,y] to [N,z,x,y]
    N = patch_data.shape[0] * patch_data.shape[1] * patch_data.shape[2]
    patch_data_merged = patch_data.reshape((N, patch_data.shape[3], patch_data.shape[4], patch_data.shape[5]))
    # segment patch_data
    patch_data_merged  = np.reshape(patch_data_merged, patch_data_merged.shape + (1,))
    result = model.predict(patch_data_merged, verbose=1, batch_size = batch_size)
    segmented = result.reshape(patch_data.shape)
    return segmented

def GMM_predict_labels(model, data, data_1d, ids_zero, mask):
    result = np.zeros(data.shape, dtype = np.uint8)
    # predict labels
    gmm_labels = model.predict(np.expand_dims(data_1d,1))
    # reshape 1D arr with labels to 3D mask 
    result[np.where(data != 0)] = gmm_labels + 1
    result = result + mask
    result[ids_zero] = 2 
    return result

In [6]:
mask = circle_mask(size = 1224, radius = 599)

In [ ]:
# load one dataset from dynamic data
data_path = '/home/m_fokin/NN/Data/data_exp2/exp2_time_19p6_101_102_107to110_113to185.h5'
f =h5.File(data_path,'r')
dsets = list(f.keys())

model = UNet_3D(input_size = (256,256,256,1))
model.load_weights("/home/m_fokin/NN/Unet_3D_2D/UNet_3D__16_256.hdf5")

# parameters
batch_size = 1
data_step = 50 
 
for dset in tqdm(dsets): 

    
    data = np.array(f[dset], dtype = np.uint8)
    ids_zero = np.where(data == 0)

    # convert data to data patches with size 256x256x256x and overlapping 128x242x242
    patch_data = patch.patchify(data, (256, 256, 256), (128, 242, 242))
    segmented_data = Unet_segmentation(patch_data, model, batch_size)

    # reconstruct data from data patches 
    recon = recon_3D(data_patches = segmented_data, patch_step = (128,242,242), patch_size = (256, 256, 256), recon_shape = (512, 1224, 1224))
    recon[recon >= 0.8] = 1 
    recon[recon < 0.8] = 0 

    # Fit GMM
    data_1d = data.ravel()
    data_1d = data_1d[data_1d != 0] 
    gmm = GaussianMixture(n_components = 4, max_iter=200, tol=1e-3, covariance_type = 'full', verbose = 0)               
    gmm = gmm.fit(X = np.expand_dims(data_1d[::data_step],1))
    
    # predict labels
    labels = GMM_predict_labels(gmm,data, data_1d , ids_zero, mask)
 
f.close()